# Introduction to FastAPI (Student Version)

**Objective**: Understand FastAPI components, routing, async/sync, parameters, and Pydantic validation.

**Note to Students**: Follow the `TP_FastAPI_Guide.md` instructions to complete the TODOs in this notebook.

## Learning Objectives
- FastAPI fundamentals and setup
- Creating routes with different HTTP methods (GET, POST, PUT, DELETE)
- Path parameters vs Query parameters
- Request/Response models with Pydantic
- Async capabilities
- Automatic API documentation (Swagger)
- REST API principles in practice

## 1. Setting Up FastAPI

First, let's install FastAPI and its dependencies. FastAPI uses Uvicorn as the ASGI server.

In [ ]:
# Import required libraries
from fastapi import FastAPI, Query, Path, HTTPException
from pydantic import BaseModel
import uvicorn
from typing import Optional
import asyncio
import threading

## 2. Creating Your First FastAPI App

FastAPI is a modern, fast web framework for building APIs. Let's create a simple app:

In [ ]:
# Create a FastAPI instance
app = FastAPI(
    title="FastAPI Learning API",
    description="A simple API to learn FastAPI concepts",
    version="1.0.0"
)

# Basic root endpoint
@app.get("/")
def read_root():
    """Welcome endpoint - returns a simple greeting"""
    # TODO: Return a simple dictionary, e.g., {"message": "Welcome to FastAPI!"}
    return {} 

def run():
    uvicorn.run(app, host="127.0.0.1", port=8000)

threading.Thread(target=run).start()

print("FastAPI server started at http://127.0.0.1:8000")

## 3. Understanding HTTP Methods (GET, POST, PUT, DELETE)

REST APIs use different HTTP methods for different actions:

In [ ]:
# Simple in-memory database for demonstration
items_db = {
    1: {"name": "Laptop", "price": 999.99, "description": "Gaming laptop"},
    2: {"name": "Mouse", "price": 29.99, "description": "Wireless mouse"}
}

# GET - Retrieve data
@app.get("/items")
def get_items():
    """Get all items"""
    return {"items": items_db, "count": len(items_db)}

@app.get("/items/{item_id}")
def get_item(item_id: int):
    """Get a specific item by ID"""
    # TODO: Check if item_id exists in items_db
    # TODO: If not, raise HTTPException with status_code 404
    # TODO: Return the item and its ID
    pass

## 4. Pydantic Models for Request/Response

Pydantic provides data validation and type hints. Let's create models:

In [ ]:
# Pydantic models for type validation and documentation
class Item(BaseModel):
    name: str
    price: float
    description: Optional[str] = None
    
class ItemResponse(BaseModel):
    id: int
    name: str
    price: float
    description: Optional[str] = None
    
class ItemUpdate(BaseModel):
    name: Optional[str] = None
    price: Optional[float] = None
    description: Optional[str] = None

# POST - Create new data
@app.post("/items", response_model=ItemResponse)
def create_item(item: Item):
    """Create a new item"""
    # TODO: Generate a new ID (e.g. max(items_db.keys()) + 1)
    # TODO: Store the item in items_db (use item.dict())
    # TODO: Return the new item with its ID
    pass

In [ ]:
# PUT - Update existing data
@app.put("/items/{item_id}", response_model=ItemResponse)
def update_item(item_id: int, item_update: ItemUpdate):
    """Update an existing item"""
    if item_id not in items_db:
        raise HTTPException(status_code=404, detail="Item not found")
    
    # TODO: Update only provided fields from item_update
    # Update stored_item with new values
    
    # return ItemResponse(id=item_id, **stored_item)
    pass

# DELETE - Remove data
@app.delete("/items/{item_id}")
def delete_item(item_id: int):
    """Delete an item"""
    # TODO: Check if item_id exists
    # TODO: Remove item from items_db
    # TODO: Return a confirmation message
    pass

print("✅ PUT and DELETE endpoints created!")
print("🔄 PUT is used to update existing resources")
print("🗑️ DELETE is used to remove resources")

## 5. Path Parameters vs Query Parameters

Understanding how to pass data to your API endpoints:

In [ ]:
# Path Parameters - part of the URL path
@app.get("/users/{user_id}/items/{item_id}")
def get_user_item(
    user_id: int = Path(..., description="ID of the user", ge=1),
    item_id: int = Path(..., description="ID of the item", ge=1)
):
    """Get an item for a specific user (using path parameters)"""
    return {
        "user_id": user_id,
        "item_id": item_id,
        "message": f"Item {item_id} for user {user_id}"
    }

# Query Parameters - after ? in the URL
@app.get("/search")
def search_items(
    q: Optional[str] = Query(None, description="Search query"),
    min_price: Optional[float] = Query(None, description="Minimum price", ge=0),
    max_price: Optional[float] = Query(None, description="Maximum price", ge=0),
    limit: int = Query(10, description="Number of results", le=100)
):
    """Search items with query parameters"""
    
    # Filter items based on query parameters
    filtered_items = items_db.copy()
    
    # TODO: Implementation needed
    # if q:
    #    Filter logic for 'name' or 'description'
    
    # if min_price is not None:
    #    Filter logic for min_price
    
    # Limit results
    limited_items = dict(list(filtered_items.items())[:limit])
    
    return {
        "query": q,
        "filters": {"min_price": min_price, "max_price": max_price},
        "limit": limit,
        "results": limited_items,
        "total_found": len(limited_items)
    }

## 6. Async vs Synchronous (ASGI vs WSGI)

FastAPI supports both sync and async operations. Here's the difference:

In [ ]:
# Synchronous endpoint (blocking)
@app.get("/sync-task")
def sync_heavy_task():
    """Synchronous task that blocks the thread"""
    import time
    time.sleep(2)  # Simulates heavy computation
    return {"message": "Sync task completed", "duration": "2 seconds"}

# Asynchronous endpoint (non-blocking)
@app.get("/async-task")
async def async_heavy_task():
    """Asynchronous task that doesn't block the thread"""
    await asyncio.sleep(2)  # Simulates async I/O operation
    return {"message": "Async task completed", "duration": "2 seconds"}

# Multiple async tasks
@app.get("/multiple-tasks")
async def multiple_async_tasks():
    """Run multiple async tasks concurrently"""
    
    async def fetch_data(task_id: int):
        await asyncio.sleep(1)  # Simulate API call
        return f"Data from task {task_id}"
    
    # Run tasks concurrently
    tasks = [fetch_data(i) for i in range(3)]
    results = await asyncio.gather(*tasks)
    
    return {
        "message": "Multiple async tasks completed",
        "results": results,
        "total_time": "~1 second (concurrent execution)"
    }

## 7. Running the FastAPI Server

Now let's run our FastAPI application and explore the automatic documentation:

In [ ]:
# Function to run the server (uncomment to run)
def run_server():
    """Run the FastAPI server"""
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")

# For demonstration, let's show what endpoints we've created
def show_routes():
    """Display all available routes"""
    routes = []
    for route in app.routes:
        if hasattr(route, 'methods') and hasattr(route, 'path'):
            routes.append({
                "path": route.path,
                "methods": list(route.methods),
                "name": route.name
            })
    return routes

print("Server endpoints:")
for route in show_routes():
    print(f"  {route['methods']} {route['path']}")

## 8. Testing Our API

Let's test some endpoints to see them in action. Ensure your TODOs are completed correctly for these tests to pass.

In [ ]:
# Test our endpoints directly (without running the server)
from fastapi.testclient import TestClient

# Create a test client
client = TestClient(app)

print("Testing API endpoints:")
print("=" * 50)

# Test GET root endpoint
response = client.get("/")
print(f"GET / -> Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test GET items
response = client.get("/items")
print(f"GET /items -> Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test GET specific item
response = client.get("/items/1")
print(f"GET /items/1 -> Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test POST - create new item
new_item = {
    "name": "Keyboard",
    "price": 79.99,
    "description": "Mechanical keyboard"
}
response = client.post("/items", json=new_item)
print(f"POST /items -> Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

# Test search with query parameters
response = client.get("/search?q=laptop&min_price=500")
print(f"GET /search?q=laptop&min_price=500 -> Status: {response.status_code}")
print(f"Response: {response.json()}")
print()

## 9. REST API Principles Summary

Let's review the key REST principles we've implemented:

### REST Principles:
1. **Resources**: Everything is a resource (items, users)
2. **HTTP Methods**: Use appropriate methods for actions
   - GET: Retrieve data
   - POST: Create new resources
   - PUT: Update existing resources
   - DELETE: Remove resources
3. **Stateless**: Each request contains all needed information
4. **Uniform Interface**: Consistent URL patterns and response formats
5. **JSON Format**: Standard data exchange format

### FastAPI Key Features:
1. **Automatic Documentation**: Swagger UI and ReDoc
2. **Type Hints**: Python type annotations for validation
3. **Pydantic Models**: Data validation and serialization
4. **Async Support**: High performance with async/await
5. **Path & Query Parameters**: Flexible data passing
6. **Error Handling**: HTTP exceptions for error responses

## 10. Next Steps & Practice Exercises

### Try These Exercises:
1. **Add a new endpoint**: Create `/health` endpoint that returns API health status
2. **Extend the Item model**: Add fields like `category`, `in_stock`, `created_date`
3. **Add validation**: Use Pydantic validators for price (must be positive) and name (min length)
4. **Create user endpoints**: Implement CRUD operations for users
5. **Add filtering**: Extend search to filter by category or stock status
6. **Error handling**: Add custom exception handlers
7. **Middleware**: Add logging middleware to track requests

### Useful Resources:
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Pydantic Documentation](https://pydantic-docs.helpmanual.io/)
- [Uvicorn Documentation](https://www.uvicorn.org/)
- [REST API Best Practices](https://restfulapi.net/)